In [1]:
import sys, os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..')))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..')))

from RFSoC_Daq import RFSoC_Daq
from Biquad import Biquad
from SimBiquad import SimBiquad
from Biquad_Daq import Biquad_Daq
from Waveforms.Waveform import Waveform

In [2]:
daq = Biquad_Daq()
sim = SimBiquad()

DEBUG:RFSoC_Daq:Asked to load overlay at /home/xilinx/python/zcumts.py
DEBUG:RFSoC_Daq:Adding directory /home/xilinx/python to module search path
DEBUG:RFSoC_Daq:Changing directory to /home/xilinx/python
DEBUG:RFSoC_Daq:Going to try to import zcumts
DEBUG:RFSoC_Daq:Found Overlay class Overlay from module pynq.overlay
DEBUG:RFSoC_Daq:Found RFSoC overlay zcuMTS


Turning on SYNC
Turning off SYNC


DEBUG:RFSoC_Daq:Created RFSoC device
DEBUG:RFSoC_Daq:Restoring original module search path
DEBUG:RFSoC_Daq:Going back to original directory /home/xilinx/rfsoc-rework/Biquad/Notebooks/tests


In [3]:
import pynq

import numpy as np
from ipywidgets import interact, interactive, fixed, interact_manual, HBox
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt

In [4]:
def update_plot(A, B, P, theta):
    
    global daq_output
    global sim_output
    global diff_output
    
    Xn = [0, 0, 0, 0, 0, 0, 0]
    Cross = [1, 0, 0, 0]
    C = [0 ,0, 0, 0]
    
    daq.update_params(A, B, P, theta*np.pi)
    
    # daq.set_Xn(*Xn)
    # daq.set_poleFIR(*Cross)
    # daq.set_poleIIR(*C)
    
    daq.quantise_coeffs()
    
    daq.run_zeroFIR()
    daq.run_poleFIR()
    daq.run_poleIIR()
    
    daq.update_waveforms()
    
    sim.update_params(A, B, P, theta*np.pi)
    
    sim.data = daq.adcBuffer[0] >> 4
    
    # sim.set_Xn(*Xn)
    # sim.set_poleFIR(*Cross)
    # sim.set_poleIIR(*C)
    
    sim.quantise_coeffs()

    sim.run_zeroFIR()
    sim.run_poleFIR()
    sim.run_poleIIR()
    
    daq_output = daq.extract_biquad()
    sim_output = sim.extract_biquad()
    diff_output = Waveform(daq_output.waveform - sim_output.waveform)
    
    fig, (axB,axS,axR) = plt.subplots(3, 1, figsize=(35, 15))
        
    daq_output.plotWaveform(axB, title="Biquaded ADC224_T0_CH0")
    
    sim_output.plotWaveform(axS, title="Simulated Biquad")
    
    diff_output.plotWaveform(axR, title="Difference")
    
    plt.show()

In [5]:
A_slider = widgets.FloatSlider(min=-8, max=7.9, step=0.05, value=1.75, description='A:', continuous_update=False)
B_slider = widgets.FloatSlider(min=-8, max=7.9, step=0.05, value=2.95, description='B:', continuous_update=False)
P_slider = widgets.FloatSlider(min=0, max=1, step=0.01, value=0.4, description='P:', continuous_update=False)
Theta_slider = widgets.FloatSlider(min=0, max=1, step=0.01, value=0.67, description='Theta:', continuous_update=False)

plot_output = widgets.Output()
    
def update(change):
    with plot_output:
        plot_output.clear_output(wait=True)
        update_plot(A = A_slider.value, B = B_slider.value, P = P_slider.value, theta = Theta_slider.value)
    
A_slider.observe(update, names='value')
B_slider.observe(update, names='value')
P_slider.observe(update, names='value')
Theta_slider.observe(update, names='value')

# Layout
slider_box = widgets.HBox([A_slider, B_slider, P_slider, Theta_slider])
ui = widgets.VBox([slider_box, plot_output])

# Display
display(ui)

# Initial plot
update(None)

In [6]:
print(daq.C0)
print(sim.C0)

-6.103515625e-05
-6.103515625e-05


In [7]:
print(np.max(sim_output))
print(np.min(sim_output))

570.0
-758.0


In [8]:
print(np.max(diff_output))
print(np.min(diff_output))
print(np.argmax(diff_output))
print(daq_output[32])
print(sim_output[32])
print(daq_output[32]-sim_output[32])

0.0
0.0
0
-93
-93.0
0.0


In [9]:
print(np.floor(10.9))

10.0
